## Detect Body Parts

Before to run this notebook run first dataset_framing notebook

To see a more detailed execution with graphical examples take a look to the openpose_example notebook

What this notebook does:

for each image:

- takes in input an image from the framed_dataset
- for each person in the image, detect body parts
- for each person in the image, build a skeleton
- save the result in a json file

In [1]:
%matplotlib inline
import cv2
import matplotlib
import pylab as plt
import numpy as np
import math
import tensorflow as tf

from models.config import get_default_configuration
from tensorflow.keras.models import load_model
from models.model import get_model

Create the keras model load the weights file

In [2]:
weights_path = "models/weights.h5"
model = get_model(1.0, 224)
model.load_weights(weights_path)

In [12]:
numFrame=0
#for numFrame....

In [16]:
path = "dataset_framing/framed_dataset/Violence/img0.jpg" #+ str(numFrame)
#path = "framed_dataset/NonViolence/img" + str(i)
oriImg = cv2.imread(path)
input_img = oriImg[np.newaxis,...] 
output_blobs = model.predict(input_img) #generate heatmaps and paf vectors
heatmap = output_blobs[3]
heatmap = heatmap[0]
paf = output_blobs[2] 
paf = paf[0]
config = get_default_configuration()
all_peaks = []
peak_counter = 0
thre1 = 0.1
hashMap = dict() 

for part_meta in config.body_parts.values():
    map = heatmap[:, :, part_meta.heatmap_idx]    
    
    map_left = np.zeros(map.shape)
    map_left[1:,:] = map[:-1,:]
    map_right = np.zeros(map.shape)
    map_right[:-1,:] = map[1:,:]
    map_up = np.zeros(map.shape)
    map_up[:,1:] = map[:,:-1]
    map_down = np.zeros(map.shape)
    map_down[:,:-1] = map[:,1:]
    
    peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up, map>=map_down, map > thre1))
    peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])) # note reverse
    peaks_with_score = [x + (map[x[1],x[0]],) for x in peaks]
    id = range(peak_counter, peak_counter + len(peaks))
    peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

    for i in id:
        hashMap[i]=part_meta.body_part.name 
        
    all_peaks.append(peaks_with_score_and_id)
    peak_counter += len(peaks)
# find connection in the specified sequence, center 29 is in the position 15
limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
           [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
           [1,16], [16,18], [3,17], [6,18]]
# the middle joints heatmap correpondence
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22], \
          [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], \
          [55,56], [37,38], [45,46]]
thre2 = 0.05

connection_all = []
special_k = []
mid_num = 10

for k in range(len(mapIdx)):
    score_mid = paf[:,:,[x-19 for x in mapIdx[k]]]
    candA = all_peaks[limbSeq[k][0]-1]
    candB = all_peaks[limbSeq[k][1]-1]
    nA = len(candA)
    nB = len(candB)
    indexA, indexB = limbSeq[k]
    if(nA != 0 and nB != 0):
        connection_candidate = []
        for i in range(nA):
            for j in range(nB):
                vec = np.subtract(candB[j][:2], candA[i][:2])
                norm = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                # failure case when 2 body parts overlaps
                if norm == 0:
                    continue
                vec = np.divide(vec, norm)
                
                startend = list(zip(np.linspace(candA[i][0], candB[j][0], num=mid_num), \
                               np.linspace(candA[i][1], candB[j][1], num=mid_num)))
                
                vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0] \
                                  for I in range(len(startend))])
                vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1] \
                                  for I in range(len(startend))])

                score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                score_with_dist_prior = sum(score_midpts)/len(score_midpts) + min(0.5*oriImg.shape[0]/norm-1, 0)
                criterion1 = len(np.nonzero(score_midpts > thre2)[0]) > 0.8 * len(score_midpts)
                criterion2 = score_with_dist_prior > 0
                if criterion1 and criterion2:
                    connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])

        connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
        connection = np.zeros((0,5))
        for c in range(len(connection_candidate)):
            i,j,s = connection_candidate[c][0:3]
            if(i not in connection[:,3] and j not in connection[:,4]):
                connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                if(len(connection) >= min(nA, nB)):
                    break

        connection_all.append(connection)
    else:
        special_k.append(k)
        connection_all.append([])
subset = -1 * np.ones((0, 20))
candidate = np.array([item for sublist in all_peaks for item in sublist])

for k in range(len(mapIdx)):
    if k not in special_k:
        partAs = connection_all[k][:,0]
        partBs = connection_all[k][:,1]
        indexA, indexB = np.array(limbSeq[k]) - 1

        for i in range(len(connection_all[k])): 
            found = 0
            subset_idx = [-1, -1]
            for j in range(len(subset)): 
                if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                    subset_idx[found] = j
                    found += 1
            
            if found == 1:
                j = subset_idx[0]
                if(subset[j][indexB] != partBs[i]):
                    subset[j][indexB] = partBs[i]
                    subset[j][-1] += 1
                    subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
            elif found == 2: # if found 2 and disjoint, merge them
                j1, j2 = subset_idx
                membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                if len(np.nonzero(membership == 2)[0]) == 0: #merge
                    subset[j1][:-2] += (subset[j2][:-2] + 1)
                    subset[j1][-2:] += subset[j2][-2:]
                    subset[j1][-2] += connection_all[k][i][2]
                    subset = np.delete(subset, j2, 0)
                else: # as like found == 1
                    subset[j1][indexB] = partBs[i]
                    subset[j1][-1] += 1
                    subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

            # if find no partA in the subset, create a new subset
            elif not found and k < 17:
                row = -1 * np.ones(20)
                row[indexA] = partAs[i]
                row[indexB] = partBs[i]
                row[-1] = 2
                row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                subset = np.vstack([subset, row])
                
# delete some rows of subset which has few parts occur
deleteIdx = [];
for i in range(len(subset)):
    if subset[i][-1] < 4 or subset[i][-2]/subset[i][-1] < 0.4:
        deleteIdx.append(i)
subset = np.delete(subset, deleteIdx, axis=0)
scale = 8

#questo print deve essere fatto nel file json, non qui
print("id Frame: " + str(numFrame))
for i in range(len(subset)):
    print("Person: " + str(i+1))
    for j in range(len(subset[i])-2):
        cella = subset[i][j]
        if cella != -1:
            X = candidate[cella.astype(int), 0] * scale
            Y = candidate[cella.astype(int), 1] * scale
            print(hashMap[cella],"(",X,"-",Y,")")

id Frame: 0
Person: 1
nose ( 160.0 - 88.0 )
neck ( 160.0 - 104.0 )
right_shoulder ( 152.0 - 104.0 )
right_elbow ( 152.0 - 128.0 )
left_shoulder ( 176.0 - 104.0 )
left_elbow ( 176.0 - 128.0 )
left_wrist ( 160.0 - 104.0 )
right_hip ( 160.0 - 136.0 )
left_hip ( 168.0 - 152.0 )
left_knee ( 160.0 - 176.0 )
left_ankle ( 160.0 - 184.0 )
right_eye ( 152.0 - 80.0 )
left_eye ( 160.0 - 80.0 )
left_ear ( 168.0 - 88.0 )
Person: 2
nose ( 104.0 - 88.0 )
neck ( 112.0 - 104.0 )
right_shoulder ( 104.0 - 104.0 )
right_elbow ( 104.0 - 128.0 )
right_wrist ( 96.0 - 136.0 )
left_shoulder ( 120.0 - 112.0 )
left_elbow ( 128.0 - 136.0 )
left_wrist ( 104.0 - 144.0 )
right_hip ( 112.0 - 152.0 )
right_knee ( 112.0 - 184.0 )
left_hip ( 120.0 - 152.0 )
left_knee ( 120.0 - 176.0 )
left_eye ( 104.0 - 88.0 )
left_ear ( 112.0 - 88.0 )
